In [ ]:
from my_functions import *

In [3]:
net_model = 'EfficientNetAutoAttB4ST'
train_db = 'DFDC'

device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')
face_policy = 'scale'
face_size = 224
frames_per_video = 128

facedet = BlazeFace().to(device)
facedet.load_weights("../blazeface/blazeface.pth")
facedet.load_anchors("../blazeface/anchors.npy")
videoreader = VideoReader(verbose=False)

In [ ]:
# Customize a Siamese NN for each public figure
names = ['RF', 'KP', 'EC', 'TB', 'JP', 'LN', 'MS', 'ND', 'CH', 'BK'] 
roc_curves_arr_old = [[] for _ in range(len(names))]
roc_curves_arr_new = [[] for _ in range(len(names))]
auc_arr = [[] for _ in range(len(names))]

flag_tune_only_last_layer = True
initial_lr = 5e-6
repeat_times = 5
epoch_num = 5000
frames_per_video = 128
num_train_video = 3

repeat_loss_arr, repeat_valid_loss_arr = [], []
eer_arr_old, eer_arr_new, eer_arr_frame_old, eer_arr_frame_new = [], [], [], []
for t_idx in range(5):
    for R in range(repeat_times):
        test_index = [t_idx]
        tmp_index = list(set(range(5)).difference(test_index))
        valid_index = sample(tmp_index, 1)
        rest_idx = list(set(tmp_index).difference(valid_index))
        train_index = sample(rest_idx, num_train_video)
        test_index2, valid_index2, train_index2 = test_index, valid_index, train_index
        
        for n in range(len(names)):
            real_videos_train, real_videos_valid, real_videos_test, fake_videos_train, fake_videos_valid, fake_videos_test = [], [], [], [], [], []
            name = names[n]
            for idx in train_index:
                with open('./saved_features/features_real_'+name+'_v_'+str(idx)+'_recon_all.pkl', 'rb') as f: res = pickle.load(f)
                real_videos_train = real_videos_train + [sample_frames_N(res, frames_per_video)]
            for idx in train_index2:
                with open('./saved_features/features_fake_'+name+'_v_'+str(idx)+'_recon_all.pkl', 'rb') as f: res = pickle.load(f)
                fake_videos_train = fake_videos_train + [sample_frames_N(res, frames_per_video)]
            for idx in valid_index:
                with open('./saved_features/features_real_'+name+'_v_'+str(idx)+'_recon_all.pkl', 'rb') as f: res = pickle.load(f)
                real_videos_valid = real_videos_valid + [sample_frames_N(res, frames_per_video)]
            for idx in valid_index2:
                with open('./saved_features/features_fake_'+name+'_v_'+str(idx)+'_recon_all.pkl', 'rb') as f: res = pickle.load(f)
                fake_videos_valid = fake_videos_valid + [sample_frames_N(res, frames_per_video)]       
            net_tuned, train_loss_history, valid_loss_history = Tune_NN(device, initial_lr, epoch_num, real_videos_train, real_videos_valid, real_videos_test, fake_videos_train, fake_videos_valid, fake_videos_test)

            res_fake_frame, res_real_frame = [], []
            real_videos_test, fake_videos_test = [], []

            for idx in test_index:
                with open('./saved_features/features_real_'+name+'_v_'+str(idx)+'_recon_all.pkl', 'rb') as f: res = pickle.load(f)
                real_videos_test = real_videos_test + [sample_frames(res)]
            for idx in test_index2:
                with open('./saved_features/features_fake_'+name+'_v_'+str(idx)+'_recon_all.pkl', 'rb') as f: res = pickle.load(f)
                fake_videos_test = fake_videos_test + [sample_frames(res) ] 
            for i in range(len(real_videos_test)):
                out1 = real_videos_test[i][0]
                out2 = real_videos_test[i][1]
                if len(out1) != len(out2):
                    tmp = min(len(out1), len(out2))
                    out1 = out1[:tmp]
                    out2 = out2[:tmp]
                out1_feature = net_tuned(out1.to(device))
                out2_feature = net_tuned(out2.to(device))
                res_real_frame = res_real_frame + list( np.sum(np.power(out1_feature.cpu().detach().numpy() - out2_feature.cpu().detach().numpy(), 2), 1) )
            for i in range(len(fake_videos_test)):
                out1 = fake_videos_test[i][0]
                out2 = fake_videos_test[i][1]
                if len(out1) != len(out2):
                    tmp = min(len(out1), len(out2))
                    out1 = out1[:tmp]
                    out2 = out2[:tmp]
                out1_feature = net_tuned(out1.to(device))
                out2_feature = net_tuned(out2.to(device))
                res_fake_frame = res_fake_frame + list( np.sum(np.power(out1_feature.cpu().detach().numpy() - out2_feature.cpu().detach().numpy(), 2), 1) )
    #         eer_frame_new, auc_frame, fp_arr_new, tp_arr_new = get_eer_result(res_real_frame,  res_fake_frame)
            Y_test =  [0] * len(res_fake_frame) + [1] * len(res_real_frame)
            false_pos_rate, true_pos_rate, _ = roc_curve(Y_test, res_fake_frame+res_real_frame)
            roc_auc = auc(false_pos_rate, true_pos_rate)
            roc_curves_arr_new[n].append([false_pos_rate, true_pos_rate])
            print('Test_v1_'+str(test_index)+', Test_v2_'+str(test_index2)+' AUC=', roc_auc)
            auc_arr[n].append(roc_auc)